In [ ]:
## Librerias
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import warnings                                              
from sklearn.exceptions import DataConversionWarning          
warnings.filterwarnings(action='ignore', category=DataConversionWarning)
pd.options.display.max_rows = None 
pd.options.display.max_columns = None

In [ ]:
df_salary = pd.read_csv('data/salaries_data.csv')
df_test = pd.read_csv('data/test.csv')
df_salary_med = pd.read_csv('data/USD_medio_pais.csv', sep=';')
df_sample = pd.read_csv('data/sample.csv', sep=',')

In [ ]:
df_salary.head(10)

In [ ]:
## Primero concatenamos el fichero donde tendremos que predecir el salario con el que ya tenemos
## De esta forma aplicaremos los mismos cambios en ambas directamente
df_salary = pd.concat([df_salary, df_test]).reset_index(drop=True)

In [ ]:
## Realizaremos una exploración de las columnas para comprender que información refleja cada una
## Si hay nulos o no y tipos de dato por columna
df_salary.info()
print('\n',df_salary.shape)
df_salary.isna().sum()

In [ ]:
df_salary.experience_level.unique()
# SE - Senior
# MI - Mid-level
# EN - Entry-level
# EX - Expert-level

In [ ]:
df_salary.employment_type.unique()
# FT - Full Time
# PT - Part Time
# FL - No compromise by any parts
# CT - Complete Time (?)

In [ ]:
df_salary[df_salary.employment_type=='CT']

In [ ]:
df_salary[df_salary.employment_type=='FL']

In [ ]:
## Comenzamos normalizando algunas de las columnas que tienen menos datos diferentes

In [ ]:
print(df_salary[df_salary.employment_type=='CT'].mean()) # CT pesa el doble que FT
print(df_salary[df_salary.employment_type=='FT'].mean())
print(df_salary[df_salary.employment_type=='FL'].mean()) # PT pesa 1.35 FL
print(df_salary[df_salary.employment_type=='PT'].mean())

In [ ]:
df_salary.employment_type = df_salary.employment_type.replace({'CT':222, 'FT':113, 'FL':48, 'PT':35})

In [ ]:
df_salary.job_title.unique() # Normalizamos

In [ ]:
## Normalizamos los tipos de trabajos dándo a cada grupo el peso de la media de salarios que contiene

In [ ]:
heads = ['principal', 'architect', 'lead', 'head', 'director', 'manager','specialist']

In [ ]:
column_aux = []
for ind, e in enumerate(df_salary.job_title):
    f = 0
    encontrado = False
    while f < len(heads) and encontrado == False:
        if heads[f] in e.lower():
            column_aux.append(15) #head
            encontrado = True
        else:
            f+=1
    if 'analyst' in e.lower() and encontrado == False:
        column_aux.append(9.4)#analyst
    elif 'engineer' in e.lower() and encontrado == False:
        column_aux.append(10.7) #engineer
    elif 'sci' in e.lower() and encontrado == False:
        column_aux.append(11.1)  #scientist
    elif 'dev' in e.lower() and encontrado == False:
        column_aux.append(6.6) # dev
    elif 'search' in e.lower() and encontrado == False:
        column_aux.append(0.5) #res

In [ ]:
df_salary.experience_level = df_salary.experience_level.replace({'SE':2, 'MI':1, 'EN':0, 'EX':4})
df_salary.employment_type = df_salary.employment_type.replace({'PT':0, 'FT':1})
df_salary.job_title = column_aux

In [ ]:
## Enriquecemos el dataframe con la media de salario para ingenieros por pais

In [ ]:
dictio = dict(zip(df_salary_med['Siglas'], df_salary_med['USD ANUAL']))
df_salary['mean_by_emp_location'] = df_salary.employee_residence.replace(dictio)
df_salary.company_size = df_salary.company_size.replace({'M':1, 'L':2, 'S':0})

In [ ]:
mascara = np.triu(np.ones_like(df_salary.corr(), dtype=bool))

sns.heatmap(df_salary.corr(), annot=True, mask = mascara)

plt.show()

In [ ]:
dictio_pais = dict((df_salary.employee_residence.value_counts(ascending=False)/df_salary.employee_residence.shape[0]*100))#[:15])

In [ ]:
df_salary.employee_residence = [dictio_pais[e] if e in dictio_pais.keys() else 0 for e in df_salary.employee_residence]
df_salary.company_location = [dictio_pais[e] if e in dictio_pais.keys() else 0 for e in df_salary.company_location]

In [ ]:
mascara = np.triu(np.ones_like(df_salary.corr(), dtype=bool))

sns.heatmap(df_salary.corr(), annot=True, mask = mascara)

plt.show()

In [ ]:
df_salary = df_salary.drop(columns=['work_year', 'employment_type', 'job_title', 'remote_ratio', 'company_size'])
df_salary.shape

In [ ]:
## Librerias ML

from sklearn.model_selection import train_test_split as tts 

from sklearn.linear_model import LinearRegression as LinReg
from sklearn.linear_model import Lasso        # regularizacion L1
from sklearn.linear_model import Ridge        # regularizacion L2
from sklearn.linear_model import ElasticNet   # regularizacion L1+L2
from sklearn.svm import SVR  # support vector regressor
from sklearn.ensemble import RandomForestRegressor as RFR  
from sklearn.tree import ExtraTreeRegressor as ETR
from sklearn.ensemble import GradientBoostingRegressor as GBR
from xgboost import XGBRegressor as XGBR
from catboost import CatBoostRegressor as CTR
from sklearn.linear_model import LogisticRegression as LogReg

## Métricas ML
from sklearn.metrics import mean_squared_error as mse  # error cuadratico medio
from sklearn.metrics import mean_absolute_error as mae  # estos alias tmb son cosa mia
from sklearn.metrics import r2_score as r2

In [ ]:
# Los 500 primeros elementos es la muestra total de donde usaremos una parte para entrenar nuestros modelos

X = df_salary[0:500].drop(columns=['salary_in_usd'])
y = df_salary[0:500].salary_in_usd

X_train, X_test, y_train, y_test = tts(X, y, train_size=0.75)#, random_state=22)

X_objetive = df_salary[500:].drop(columns=['salary_in_usd']) # Hay que predecir sobre esta parte
#y_test = df_salary[500:].salary_in_usd

In [ ]:
linreg=LinReg()
lasso=Lasso()
ridge=Ridge()
elastic=ElasticNet()
svr=SVR()
rfr=RFR()
etr=ETR()
gbr=GBR()
xgbr=XGBR()
ctr=CTR()

nombres = ['linreg', 'lasso', 'ridge', 'elastic', 'svr', 'rfr', 'etr', 'gbr', 'xgbr', 'ctr']
modelos = []
modelos.append(linreg)
modelos.append(lasso)
modelos.append(ridge)
modelos.append(elastic)
modelos.append(svr)
modelos.append(rfr)
modelos.append(etr)
modelos.append(gbr)
modelos.append(xgbr)
modelos.append(ctr)

In [ ]:
results = []
for ind,modelo in enumerate(modelos):
    aux=[]
    if nombres[ind]=='ctr':
        modelo.fit(X_train, y_train, verbose=0)
    else:
        modelo.fit(X_train, y_train)
        
    y_pred = modelo.predict(X_test)
    y_pred_train = modelo.predict(X_train)
    
    aux.append(nombres[ind])
    aux.append(mse(y_test, y_pred, squared=False))
    aux.append(mse(y_train, y_pred_train, squared=False))
    aux.append(modelo.score(X_train, y_train))
    aux.append(modelo.score(X_test, y_test))
    
    results.append(aux)

In [ ]:
# Buscaremos el modelo que menor RMSE presente teniendo en cuenta el score en segundo plano
display(results)

In [ ]:
# Optamos por el GBR 
ctr=CTR()
ctr.fit(X, y)
y_pred = ctr.predict(X_objetive)

In [ ]:
ctr.score(X, y)

In [ ]:
y_pred

In [ ]:
len(y_pred)

In [ ]:
df_sample.salary_in_usd = y_pred
df_sample.to_csv('data/result.csv', index=False)